# Step 1: mount the google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
#Perhaps this step can be skipped by saving directly to the workspace

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Step 2: install the packages

In [ ]:
#package instalation

#Installing PROSAIL
!pip install prosail

#latin hypercube stuff
#lets try to do a LHS
!pip install lhsmdu

#this package as a number of functions to deal with hyperspectral data
!pip install pysptools

     |████████████████████████████████| 153kB 2.7MB/s 
     |████████████████████████████████| 8.1MB 2.8MB/s 
  Created wheel for pysptools: filename=pysptools-0.15.0-cp36-none-any.whl size=8133750 sha256=c9704c7d2221fb3158d4336a0441ad98af064578e2adf02f9eb6d50ac2580aad
  Stored in directory: /root/.cache/pip/wheels/bf/60/be/a6719d91bfa59135201feb034c7069e4146aa576fc0dc9e624
Successfully built pysptools


#TensorFlow update correction

Tensor flow has been updated so we will uninstall the new version and push in the old version which works well on my case

In [ ]:
#!pip uninstall tensorflow

Uninstalling tensorflow-2.2.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.6/dist-packages/tensorflow-2.2.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/tensorflow/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-2.2.0


In [ ]:
#!pip install tensorflow==1.14.0

     |████████████████████████████████| 109.2MB 92kB/s 
     |████████████████████████████████| 491kB 38.1MB/s 
     |████████████████████████████████| 3.2MB 39.4MB/s 
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
  Found existing installation: tensorboard 2.2.1
    Uninstalling tensorboard-2.2.1:
      Successfully uninstalled tensorboard-2.2.1


# Step 3 Import the packages

In [ ]:
#General purpose: 
import matplotlib.pyplot as plt
import numpy
import numpy as np
from mpl_toolkits.mplot3d import Axes3D

#the beutiful R like data frame
import pandas as pd

#PROSPECT+SAIL Radiative transfer mode package
import prosail

#Sampling design package
import lhsmdu

#a few more stuff for random
import random as rdm
import math


#package to for operations on spectral data
import pysptools as sptool 
from pysptools import distance


from sklearn.model_selection import KFold # import KFold

#machine learning stuff
#NEURAL NETWORK - Keras will be updated soon so this colab will also have to be changed

from sklearn.neural_network import MLPRegressor as ANN_reg #this is a simpler neural network package
from keras.models import Sequential, Model
from keras.layers import Dense, Input
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
#ignore the warning for now

#Random FOREST
# Fitting Random Forest Regression to the dataset 
# import the regressor 
from sklearn.ensemble import RandomForestRegressor 

#Gaussian processes
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import ConstantKernel, Matern, WhiteKernel, ExpSineSquared, RationalQuadratic, RBF

#aux functions
from sklearn.preprocessing import MinMaxScaler #this is to standardize the input data [not used for now]
from sklearn import metrics

#for model storing -sklearn
import pickle

#for model storing keras
from keras.models import model_from_json

Using TensorFlow backend.


Instructions for updating:
non-resource variables are not supported in the long term


# Step 4.1: set up of custom auxiliar functions

- First we have to set of custom function to facilitate the loop calls



In [ ]:
def custom_prosail(cab,cw,cm,lai):
  import prosail
  #default parameters
  n= 1.
  car=10.
  cbrown=0.01
  typelidf=1 #this is the default option
  lidfa = -1 #leaf angle distribution parameter a and b
  lidfb=-0.15
  hspot= 0.01 #hotspot parameters - got this from R package https://www.rdocumentation.org/packages/hsdar/versions/0.4.1/topics/PROSAIL
  #sun and viewing angle
  tts=30. #observation and solar position parameters
  tto=10. 
  psi=0.
  #for now i put them by hand but they should be an input of a custom function
  #tts=sol_zen #solar zenith angle
  #tto=inc_zen #sensor zenith angle
  #psi=raa
  rho_out = prosail.run_prosail(n,
                                 cab,
                                 car,
                                 cbrown,
                                 cw,
                                 cm,
                                 lai,
                                 lidfa,
                                 hspot,
                                 tts,tto,psi,
                                 typelidf, lidfb,
                                 prospect_version="D",
                                 factor='SDR', 
                                 rsoil=.5, psoil=.5)
  return(rho_out)

def Prosail2S2(path2csv,spectra_input):
  #importing pandas
  import pandas as pd
  import numpy
  import numpy as np
  #upload a S2_Response.csv from https://earth.esa.int/web/sentinel/user-guides/sentinel-2-msi/document-library/-/asset_publisher/Wk0TKajiISaR/content/sentinel-2a-spectral-responses

  s2_table = pd.read_csv(path2csv,sep=";",decimal=",") #check if this is proper, regarding the sep and dec
  #chekc which row you are actually extracting

  s2_table_sel = s2_table[s2_table['SR_WL'].between(400,2500)] #selects all values between 400 and 2500
  spectra_input_df = pd.DataFrame(data=spectra_input,columns=["rho"],index=s2_table_sel.index) #transforms the input array into a pandas df with the column name rho and row.index = to the original input table

  
  rho_s2 = s2_table_sel.multiply(spectra_input_df['rho'],axis="index") #calculates the numerator
  w_band_sum = s2_table_sel.sum(axis=0,skipna = True) #calculates the denominator

  output = (rho_s2.sum(axis=0)/w_band_sum).rename_axis("ID").values #runs the weighted mean and converts the output to a numpy array

  return output[1:] #removes the first value because it represents the wavelength column

#please LOAD THTE FILE NOW
filepath="/content/drive/My Drive/S2_Response.csv"
#filepath="/content/S2_Responses_S2B.csv"
#filepath="/content/drive/My Drive/S2_Response.csv"


def Gen_spectra_data(traits):
  k = 1
  #pd_train_traits=traits
  #print(range(len(traits)))
  for i in range(len(traits)):
    #n_t = pd_train_traits["n"][i]
    cab_t = traits["cab"][i]
    #car_t = pd_train_traits["car"][i]
    #cbrown_t = pd_train_traits["cbrown"][i]
    cw_t = traits["cw"][i]
    cm_t = traits["cm"][i]
    lai_t = traits["lai"][i]

    if k == 1:
      tr_rho_s = custom_prosail(cab_t,cw_t,cm_t,lai_t)
      tr_rho_s = Prosail2S2(filepath,tr_rho_s)
      #plt.plot ( x, tr_rho_s, ':', label="Training prosail")
      #plt.legend(loc='best')
      
    if k > 1:
      tr_rho_t = custom_prosail(cab_t,cw_t,cm_t,lai_t)
      tr_rho_t = Prosail2S2(filepath,tr_rho_t)
      tr_rho_s = np.vstack((tr_rho_s,tr_rho_t))
      #plt.plot ( x, tr_rho_t, ':')

    k = k+1


  rho_samples=tr_rho_s


  return rho_samples




#Function to store the outputs into a table
column_names=["Model",
              "NSamples",
              "OutOfBag",
              "KFold_tr_samples",
              "KFold_vl_samples",
              "Variable",
              "Fold_nr",
              "ExplVar",
              "Max_err",
              "Mean_abs_Err",
              "Mean_sqr_err",
              #"Mean_sqr_lg_err",
              "Median_abs_err",
              "r2",
              "MAPE",
              "ModelName"]
              #"Mean_poiss_dev",
              #"Mean_gamma_dev"]
              #"Mean_tweed_dev"]

#mape is not existant in the package so we have to create it:
#https://stats.stackexchange.com/questions/58391/mean-absolute-percentage-error-mape-in-scikit-learn
#from sklearn.utils import check_array
def mean_absolute_percentage_error(y_true, y_pred): 

    ## Note: does not handle mix 1d representation
    #if _is_1d(y_true): 
    #    y_true, y_pred = _check_1d_array(y_true, y_pred)

    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

  #Here, the file is used for saving
  #creating a df to receive the data


def calc_metrics(MDL,Samples,oob_samples,kf_tr,kf_vl,Variable,Fold,Ref,Pred,Modelname):

  out_list = {"Model":MDL,
              "NSamples":Samples,
              "OutOfBag":oob_samples,
              "KFold_tr_samples":kf_tr,
              "KFold_vl_samples":kf_vl,
              "Variable":Variable,
              "Fold_nr":Fold,
              "ExplVar": metrics.explained_variance_score(Ref, Pred),
              "Max_err": metrics.max_error(Ref, Pred),
              "Mean_abs_Err": metrics.mean_absolute_error(Ref, Pred),
              "Mean_sqr_err": metrics.mean_squared_error(Ref, Pred),
              "Median_abs_err" : metrics.median_absolute_error(Ref, Pred),
              "r2": metrics.r2_score(Ref, Pred),
              "MAPE": mean_absolute_percentage_error(Ref, Pred),
              "ModelName":Modelname}


  return out_list

# Step 4.2: Setting up machine learning models 


In [ ]:
#Random forest and GPR
from sklearn.gaussian_process.kernels import RBF,DotProduct,Matern,ExpSineSquared,RationalQuadratic
#Random forest
#initializing the random forest
#optimized

#{'min_samples_leaf': 0.0007218586390063501, 'min_samples_split': 0.0006729906646919075, 'n_estimators': 17} 800
def rfr_call():

  mdl = RandomForestRegressor(n_estimators=850,random_state=0,
                               min_samples_leaf=0.0007218586390063501,
                               min_samples_split=0.0006729906646919075,
                               verbose=0) 
  return mdl

rfr_ml = rfr_call()


#gaussian processes with default acquisition function
#initializing the gaussian process -optimized
K_ratQ = RationalQuadratic(length_scale_bounds="fixed",alpha_bounds="fixed")

#{'kernel': 1, 'n_restarts_optimizer': 7, 'normalize_y': 0}
def gpr_call():
  mdl = GaussianProcessRegressor(kernel=K_ratQ,
                                  n_restarts_optimizer=80, # -> 1*RBF(1.0) is the default kerneºl
                                  normalize_y=True,
                                  random_state=0)
  
  return mdl

gpr_ml = gpr_call()

In [ ]:
#parameters dictonary to build up the networks
hiddenlayer = {0:'one',1:'two',2:'three'}
modeltype = {0: 'MTST',1: 'MTSTST',2: 'MTMTST',3: 'MTMTSTST'}
activation_f = {0:'linear', 1:'sigmoid', 2:'tanh',3:'exponential',4:'softplus',5:'relu',6:'softsign'}
units_shared = {0: 5,1: 10,2: 15,3: 20}
units_traits = {0:1,1:2,2:3,3:4,4:5,5:6,6:7,7:8,8:9,9:10}
optimizer = {0: 'adam',1: 'RMSprop',2: 15,3: 'Adadelta'}

#for the single task network
param_dict_st_ann = {'aa_hiddenlayers':hiddenlayer[2],
              'activation_3_1': activation_f[6],
              'activation_3_2': activation_f[6],
              'activation_3_3': activation_f[5],
              'optimizer_3': optimizer[0],
              'units_3_1': units_shared[3],
              'units_3_2': units_shared[3],
              'units_3_3': units_shared[2]}
param_dict_st_ann
#for the multi-task network
param_dict_mtn = {'aaa_modeltype':modeltype[2],
              'activation_MTMTST':activation_f[2],
              'activation_MTMTST2': activation_f[5],
              'activation_cab_MTMTST': activation_f[6],'activation_cm_MTMTST': activation_f[0],
              'activation_cw_MTMTST': activation_f[6],'activation_lai_MTMTST': activation_f[2],
              'optimizer_MTMTST': optimizer[0],
              'units_cab_MTMTST': units_traits[7],'units_cm_MTMTST': units_traits[6],
              'units_cw_MTMTST': units_traits[7],'units_lai_MTMTST': units_traits[8],
              'units_shared_MTMTST': units_shared[3],
              'units_shared_MTMTST2': units_shared[3]}


#### setting up ANN
def st_ann(params):

  from keras.models import Sequential
  from keras.layers.core import Dense, Activation
  from keras.optimizers import Adadelta, Adam, rmsprop

  mdl = Sequential()
  #mdl.add(Dense(units=params['units_1'],input_dim=9,activation=params['activation_1']))
  #mdl.add(Activation(params['activation_1']))
  #print("This is the dictionary")
  #print(params)

  hiddenlayers = units=params['aa_hiddenlayers']
  
  if hiddenlayers == 'one':
    mdl.add(Dense(units=params['units_1_1'],input_dim=9,activation=params['activation_1_1']))

    #and the output layer
    mdl.add(Dense(4,activation='linear'))
    #mdl.add(Activation("linear"))
    mdl.compile(loss='mean_squared_error', optimizer=params['optimizer_1'], metrics=['mean_squared_error'])
  
  if hiddenlayers == 'two':
    mdl.add(Dense(units=params['units_2_1'],input_dim=9,activation=params['activation_2_1']))
    mdl.add(Dense(units=params['units_2_2'],activation=params['activation_2_2']))

    #and the output layer
    mdl.add(Dense(4,activation='linear'))
    #mdl.add(Activation("linear"))
    mdl.compile(loss='mean_squared_error', optimizer=params['optimizer_2'], metrics=['mean_squared_error'])
    
  if hiddenlayers == 'three':
    mdl.add(Dense(units=params['units_3_1'],input_dim=9,activation=params['activation_3_1']))
    mdl.add(Dense(units=params['units_3_2'],activation=params['activation_3_2']))
    mdl.add(Dense(units=params['units_3_3'],activation=params['activation_3_3']))

    #and the output layer
    mdl.add(Dense(4,activation='linear'))
    #mdl.add(Activation("linear"))
    mdl.compile(loss='mean_squared_error', optimizer=params['optimizer_3'], metrics=['mean_squared_error'])


  return mdl


#### setting up MTN
def mt_ann(params):

  #mdl = Sequential()
  #mdl.add(Dense(units=params['units_1'],input_dim=9,activation=params['activation_1']))
  #mdl.add(Activation(params['activation_1']))
  #print("This is the dictionary")
  #print(params)

  inputs = Input(shape=(9,))

  #model selector
  struct = params['aaa_modeltype']
  if struct == 'MTST':
    shared_MTST = Dense(units=params['units_shared_MTST'],activation=params['activation_MTST'])(inputs)
    cab_MTST = Dense(units=params['units_cab_MTST'], activation=params['activation_cab_MTST'])(shared_MTST)
    cw_MTST  = Dense(units=params['units_cw_MTST'],  activation=params['activation_cw_MTST'])(shared_MTST)
    cm_MTST  = Dense(units=params['units_cm_MTST'],  activation=params['activation_cm_MTST'])(shared_MTST)
    lai_MTST = Dense(units=params['units_lai_MTST'], activation=params['activation_lai_MTST'])(shared_MTST)

    #this is a just a final output layer needed for the whole thing to work
    cab_out = Dense(1, activation='linear')(cab_MTST)
    cw_out  = Dense(1, activation='linear')(cw_MTST)
    cm_out  = Dense(1, activation='linear')(cm_MTST)
    lai_out = Dense(1, activation='linear')(lai_MTST)

    mdl = Model(inputs=inputs, outputs=[cab_out,cw_out,cm_out,lai_out])
    mdl.compile(loss='mean_squared_error', optimizer=params['optimizer_MTST'], metrics=['mean_squared_error'])


  if struct == 'MTSTST':
    shared_MTSTST = Dense(units=params['units_shared_MTSTST'],activation=params['activation_MTSTST'])(inputs)
    cab_MTSTST = Dense(units=params['units_cab_MTSTST1'], activation=params['activation_cab_MTSTST1'])(shared_MTSTST)
    cw_MTSTST  = Dense(units=params['units_cw_MTSTST1'],  activation=params['activation_cw_MTSTST1'])(shared_MTSTST)
    cm_MTSTST  = Dense(units=params['units_cm_MTSTST1'],  activation=params['activation_cm_MTSTST1'])(shared_MTSTST)
    lai_MTSTST = Dense(units=params['units_lai_MTSTST1'], activation=params['activation_lai_MTSTST1'])(shared_MTSTST)

    cab_MTSTST2 = Dense(units=params['units_cab_MTSTST2'], activation=params['activation_cab_MTSTST2'])(cab_MTSTST)
    cw_MTSTST2  = Dense(units=params['units_cw_MTSTST2'],  activation=params['activation_cw_MTSTST2'])(cw_MTSTST)
    cm_MTSTST2  = Dense(units=params['units_cm_MTSTST2'],  activation=params['activation_cm_MTSTST2'])(cm_MTSTST)
    lai_MTSTST2 = Dense(units=params['units_lai_MTSTST2'], activation=params['activation_lai_MTSTST2'])(lai_MTSTST)

    #this is a just a final output layer needed for the whole thing to work
    cab_out = Dense(1, activation='linear')(cab_MTSTST2)
    cw_out  = Dense(1, activation='linear')(cw_MTSTST2)
    cm_out  = Dense(1, activation='linear')(cm_MTSTST2)
    lai_out = Dense(1, activation='linear')(lai_MTSTST2)

    mdl = Model(inputs=inputs, outputs=[cab_out,cw_out,cm_out,lai_out])
    mdl.compile(loss='mean_squared_error', optimizer=params['optimizer_MTSTST'], metrics=['mean_squared_error'])

  if struct == 'MTMTST':
    shared_MTMTST = Dense(units=params['units_shared_MTMTST'],activation=params['activation_MTMTST'])(inputs)
    shared_MTMTST2 = Dense(units=params['units_shared_MTMTST2'],activation=params['activation_MTMTST2'])(shared_MTMTST)

    cab_MTMTST = Dense(units=params['units_cab_MTMTST'], activation=params['activation_cab_MTMTST'])(shared_MTMTST2)
    cw_MTMTST  = Dense(units=params['units_cw_MTMTST'],  activation=params['activation_cw_MTMTST'])(shared_MTMTST2)
    cm_MTMTST  = Dense(units=params['units_cm_MTMTST'],  activation=params['activation_cm_MTMTST'])(shared_MTMTST2)
    lai_MTMTST = Dense(units=params['units_lai_MTMTST'], activation=params['activation_lai_MTMTST'])(shared_MTMTST2)

    #this is a just a final output layer needed for the whole thing to work
    cab_out = Dense(1, activation='linear')(cab_MTMTST)
    cw_out  = Dense(1, activation='linear')(cw_MTMTST)
    cm_out  = Dense(1, activation='linear')(cm_MTMTST)
    lai_out = Dense(1, activation='linear')(lai_MTMTST)

    mdl = Model(inputs=inputs, outputs=[cab_out,cw_out,cm_out,lai_out])
    mdl.compile(loss='mean_squared_error', optimizer=params['optimizer_MTMTST'], metrics=['mean_squared_error'])

  if struct == 'MTMTSTST':
    shared_MTMTSTST = Dense(units=params['units_shared_MTMTSTST'],activation=params['activation_MTMTSTST'])(inputs)
    shared_MTMTSTST2 = Dense(units=params['units_shared_MTMTSTST2'],activation=params['activation_MTMTSTST2'])(shared_MTMTSTST)

    cab_MTMTSTST = Dense(units=params['units_cab_MTMTSTST1'], activation=params['activation_cab_MTMTSTST1'])(shared_MTMTSTST2)
    cw_MTMTSTST  = Dense(units=params['units_cw_MTMTSTST1'],  activation=params['activation_cw_MTMTSTST1'])(shared_MTMTSTST2)
    cm_MTMTSTST  = Dense(units=params['units_cm_MTMTSTST1'],  activation=params['activation_cm_MTMTSTST1'])(shared_MTMTSTST2)
    lai_MTMTSTST = Dense(units=params['units_lai_MTMTSTST1'], activation=params['activation_lai_MTMTSTST1'])(shared_MTMTSTST2)

    cab_MTMTSTST2 = Dense(units=params['units_cab_MTMTSTST2'], activation=params['activation_cab_MTMTSTST2'])(cab_MTMTSTST)
    cw_MTMTSTST2  = Dense(units=params['units_cw_MTMTSTST2'],  activation=params['activation_cw_MTMTSTST2'])(cw_MTMTSTST)
    cm_MTMTSTST2  = Dense(units=params['units_cm_MTMTSTST2'],  activation=params['activation_cm_MTMTSTST2'])(cm_MTMTSTST)
    lai_MTMTSTST2 = Dense(units=params['units_lai_MTMTSTST2'], activation=params['activation_lai_MTMTSTST2'])(lai_MTMTSTST)

    #this is a just a final output layer needed for the whole thing to work
    cab_out = Dense(1, activation='linear')(cab_MTMTSTST2)
    cw_out  = Dense(1, activation='linear')(cw_MTMTSTST2)
    cm_out  = Dense(1, activation='linear')(cm_MTMTSTST2)
    lai_out = Dense(1, activation='linear')(lai_MTMTSTST2)

    mdl = Model(inputs=inputs, outputs=[cab_out,cw_out,cm_out,lai_out])
    mdl.compile(loss='mean_squared_error', optimizer=params['optimizer_MTMTSTST'], metrics=['mean_squared_error'])

  return mdl


#Training epochs
number_epochs = 500

# Step 4.3: Trait-space

In [ ]:
n_traits=4 #I will test on 4 varying traits: cab, car, cw,cm,lai

#generating a LHS hypercube (it uses a 0 to 1 interval that can be used as a multiplier against the different traits)
np.random.seed(0)

#the values here are 1 less than the max so i can add a value later to it
#max_n=1 
max_cab=121. #add 1
#max_car=44. #add 1
#max_cbrown= 9.99 #add 0.01
max_cw=0.008 #add 0.001 
max_cm=0.008 #0.001
max_lai = 9.9 #add 0.1

min_cab = 10.
min_cw = 0.002
min_cm = 0.002
min_lai = .5

#step 5: setting up the Loop

In [ ]:
#setting up the limits of the run

#minimium samples
n_samples = 100
#n_points = np.arange(4000,n_samples-1,-250)
#n_points= np.append(n_points,150)
#n_points= np.append(n_points,50)
#n_points = np.arange(3300,4000+1,100)
#n_points = [50,100,150,200,250,300,400,500,600,700,800,900,1000,]
#n_points = [50,150]
#n_points = np.append(n_points,np.arange(n_samples,4000,250))
#n_points= np.append(n_points,4000)
#n_points= np.append(n_points,50)
n_points = [50,100, 150, 200, 250, 350, 500, 750, 1000, 1250, 1500, 1750, 2000, 2500, 3000, 3500, 4000]
n_points = [1750, 2000, 2500, 3000, 3500, 4000]

print("Loop for the following set of LHS samples:",n_points)


Loop for the following set of LHS samples: [1750, 2000, 2500, 3000, 3500, 4000]


#step 6: The loop


In [ ]:
### storing the dataset

#######################################################
#BEWARE - > Check the paths for file saving carefully
########################################################

#this is for the training dataset
df_metrics = pd.DataFrame(columns=column_names)
#this is for the out of bag
df_metrics_valid = pd.DataFrame(columns=column_names)

#a path to store the models 
#path2folder = "/content/drive/My Drive/DSRP_Lunch_outputs/Models/"
path2folder = "/content/drive/My Drive/RTM_Inversion/Pure/Run_02/Models"

#minimium samples
#n_samples = 500
#n_points = np.arange(n_samples,6000+1,500)

print("Loop for the following set of LHS samples:",n_points)

#this command is enough to set u the k-fold
fold_nr=5
kf = KFold(n_splits=fold_nr,shuffle=True,random_state=0) # Define the split 

#this next comment snippet shouldn't be needed anymore

##here we generate the out of bag test dataset of 1000 points that is used as out-of-bag testing
#LHS_valid = lhsmdu.createRandomStandardUniformMatrix(n_traits,1000)
#valid_pd_trait = pd.DataFrame.transpose(pd.DataFrame(LHS_valid))
#valid_pd_trait.columns = ["cab","cw","cm","lai"]

#valid_pd_trait["cab"]=valid_pd_trait["cab"]*max_cab+min_cab
#valid_pd_trait["cw"] =valid_pd_trait["cw"] *max_cw+min_cw
#valid_pd_trait["cm"] =valid_pd_trait["cm"] *max_cm+min_cw
#valid_pd_trait["lai"]=valid_pd_trait["lai"]*max_lai+min_lai

#valid_np_trait = valid_pd_trait.iloc[:,:].values
#valid_np_spect = Gen_spectra_data(valid_pd_trait)
#valid_df_spect = valid_np_spect[:,[1,2,3,4,5,6,8,11,12]]

for j in n_points:
  print("Generating trait table for",j,"samples")

  #this section generates trait space - careful with the column names as this is significant later
  LHS_train = lhsmdu.createRandomStandardUniformMatrix(n_traits,j)
  pd_trait = pd.DataFrame.transpose(pd.DataFrame(LHS_train))
  pd_trait.columns = ["cab","cw","cm","lai"]

  pd_trait["cab"]=pd_trait["cab"]*max_cab+min_cab
  pd_trait["cw"] =pd_trait["cw"] *max_cw+min_cw
  pd_trait["cm"] =pd_trait["cm"] *max_cm+min_cw
  pd_trait["lai"]=pd_trait["lai"]*max_lai+min_lai

  #lets also create a numpy object for the traits - this is for ease of operation later
  np_trait = pd_trait.iloc[:,:].values

  #until here its instantaneous

  #now we first generate the date in hyperspectral, convolute it to S2 while iterating through the entire given trait space
  np_spect = Gen_spectra_data(pd_trait)[:,[1,2,3,4,5,6,8,11,12]]
  #trait_df = np_spectra

  #now from these generated samples, we take out 10% of the points (aproximately, sometimes its a decimal more, depends on the number)
  #creating row identifiers
  #this gets the length for the entire list
  index = list(range(len(np_spect)))

  index10 = rdm.sample(index,math.ceil(len(index)*.1)) #randomly selects 10% of te data (aproximately)
  index90 = [x for x in index if x not in index10] #makes a list set wit the reamining
  #once we have the list of point, we can subset

  vl_trait_df = np_trait[index10,]
  tr_trait_df = np_trait[index90,]
  
  vl_spect_df = np_spect[index10,]
  tr_spect_df = np_spect[index90,]

  #this is a function to re-scale inputs for the neural networks
  scaler = MinMaxScaler()

  print("Of which, 10% were taken as out-of-bag:", len(vl_trait_df))

  #now we have to iterate per fold, train and store the prediction
  k=1
  for train_index, test_index in kf.split(tr_spect_df):

    print("processing fold nr: " + str(k))

    #print("Calculating fold ",k,"of",fold_nr)
    X_train, X_test = tr_spect_df[train_index], tr_spect_df[test_index]
    Y_train, Y_test = tr_trait_df[train_index], tr_trait_df[test_index]
    label_names = ["cab","cw","cm","lai"]

    #restarting the models
    rfr_ml = rfr_call()
    gpr_ml = gpr_call()
    ann_ml = st_ann(param_dict_st_ann)
    mtn_ml = mt_ann(param_dict_mtn)

    #training the machine learning models
    #random forest
    rfr_ml.fit(X_train,Y_train)
    #shallow multi objective single task neural network
    scaler.fit(Y_train)
    Y_train_norm = scaler.transform(Y_train)
    ann_ml.fit(X_train,Y_train_norm,epochs=number_epochs,verbose=0)

    #GPR - Training 
    gpr_ml.fit(X_train,Y_train)

    #"Shallow" Multi task ANN training
    cab_train = Y_train_norm[:,0]
    cw_train = Y_train_norm[:,1]
    cm_train = Y_train_norm[:,2]
    lai_train = Y_train_norm[:,3]
    mtn_ml.fit(X_train,[cab_train,cw_train,cm_train,lai_train],epochs=number_epochs,verbose=0)#hides spam on the output


    #Prediction on training fold
    y_ann = scaler.inverse_transform(ann_ml.predict(X_test))
    y_rfr = rfr_ml.predict(X_test)
    y_gpr = gpr_ml.predict(X_test)

    #the multi task is a bit more complicated
    Y_mtn = mtn_ml.predict(X_test)
    cab_pred = Y_mtn[0]
    cw_pred  = Y_mtn[1]
    cm_pred  = Y_mtn[2]
    lai_pred = Y_mtn[3]
    np_Y_ann_pred = np.array(cab_pred)
    np_Y_ann_pred = np.hstack((np_Y_ann_pred,cw_pred))
    np_Y_ann_pred = np.hstack((np_Y_ann_pred,cm_pred))
    np_Y_ann_pred = np.hstack((np_Y_ann_pred,lai_pred))

    y_mtn = scaler.inverse_transform(np_Y_ann_pred)

    #prediction on out of bag
    y_ann_valid = scaler.inverse_transform(ann_ml.predict(vl_spect_df))
    y_rfr_valid = rfr_ml.predict(vl_spect_df)
    y_gpr_valid = gpr_ml.predict(vl_spect_df)

    Y_mtn_valid = mtn_ml.predict(vl_spect_df)
    cab_pred_valid = Y_mtn_valid[0]
    cw_pred_valid  = Y_mtn_valid[1]
    cm_pred_valid  = Y_mtn_valid[2]
    lai_pred_valid = Y_mtn_valid[3]
    np_Y_ann_pred_valid = np.array(cab_pred_valid)
    np_Y_ann_pred_valid = np.hstack((np_Y_ann_pred_valid,cw_pred_valid))
    np_Y_ann_pred_valid = np.hstack((np_Y_ann_pred_valid,cm_pred_valid))
    np_Y_ann_pred_valid = np.hstack((np_Y_ann_pred_valid,lai_pred_valid))

    y_mtn_valid = scaler.inverse_transform(np_Y_ann_pred_valid)


    #setting the model filenames
    #zfill is a handy function to standardize the output, i love it!
    # rfr_ml_modelname = path2folder+"RFR"+"_S"+str(j).zfill(5)+"_K"+str(k).zfill(3)+".pkl"
    # gpr_ml_modelname = path2folder+"GPR"+"_S"+str(j).zfill(5)+"_K"+str(k).zfill(3)+".pkl"
    # ann_ml_modelname = path2folder+"ANN"+"_S"+str(j).zfill(5)+"_K"+str(k).zfill(3)+".h5"
    # mtn_ml_modelname = path2folder+"MTN"+"_S"+str(j).zfill(5)+"_K"+str(k).zfill(3)+".h5"
    rfr_ml_modelname = "Not saved"
    gpr_ml_modelname = "Not saved"
    ann_ml_modelname = "Not saved"
    mtn_ml_modelname = "Not saved"


    #COMMENT HERE IF YOU DO NOT WANT TO SAVE THE MODELS
    #saving the models
    #sklearn are pickles while keras are h5 files
    #not saving random forest for now since i am pretty sure it wont be good but it is a big file...
    #with open(rfr_ml_modelname, 'wb') as file:
    #  pickle.dump(rfr_ml, file)

    #with open(gpr_ml_modelname, 'wb') as file:
    #  pickle.dump(gpr_ml, file)

    #ann_ml.save(ann_ml_modelname)
    #model.save(mtn_ml_modelname)
      


    #the next section stores all the results
    for i in range(n_traits):

      #this stores the training accuraccy metrics
      #print(i)
      ann_temp_list = calc_metrics("ANN",j,len(vl_spect_df),len(X_train),len(X_test),label_names[i],k,Y_test[:,i],y_ann[:,i],ann_ml_modelname)
      rfr_temp_list = calc_metrics("RFR",j,len(vl_spect_df),len(X_train),len(X_test),label_names[i],k,Y_test[:,i],y_rfr[:,i],rfr_ml_modelname)
      gpr_temp_list = calc_metrics("GPR",j,len(vl_spect_df),len(X_train),len(X_test),label_names[i],k,Y_test[:,i],y_gpr[:,i],gpr_ml_modelname)
      mtn_temp_list = calc_metrics("MTN",j,len(vl_spect_df),len(X_train),len(X_test),label_names[i],k,Y_test[:,i],y_mtn[:,i],mtn_ml_modelname)

      #This stores the metrics against the out of bag list
      ann_temp_list_valid = calc_metrics("ANN",j,len(vl_spect_df),len(X_train),len(X_test),label_names[i],k,vl_trait_df[:,i],y_ann_valid[:,i],ann_ml_modelname)
      rfr_temp_list_valid = calc_metrics("RFR",j,len(vl_spect_df),len(X_train),len(X_test),label_names[i],k,vl_trait_df[:,i],y_rfr_valid[:,i],rfr_ml_modelname)
      gpr_temp_list_valid = calc_metrics("GPR",j,len(vl_spect_df),len(X_train),len(X_test),label_names[i],k,vl_trait_df[:,i],y_gpr_valid[:,i],gpr_ml_modelname)
      mtn_temp_list_valid = calc_metrics("MTN",j,len(vl_spect_df),len(X_train),len(X_test),label_names[i],k,vl_trait_df[:,i],y_mtn_valid[:,i],mtn_ml_modelname)

      #appending to the dataframe (training error)
      df_metrics = df_metrics.append(ann_temp_list,ignore_index=True)
      df_metrics = df_metrics.append(rfr_temp_list,ignore_index=True)
      df_metrics = df_metrics.append(gpr_temp_list,ignore_index=True)
      df_metrics = df_metrics.append(mtn_temp_list,ignore_index=True)

      #appending to the dataframe (out of bag error)
      df_metrics_valid = df_metrics_valid.append(ann_temp_list_valid,ignore_index=True)
      df_metrics_valid = df_metrics_valid.append(rfr_temp_list_valid,ignore_index=True)
      df_metrics_valid = df_metrics_valid.append(gpr_temp_list_valid,ignore_index=True)
      df_metrics_valid = df_metrics_valid.append(mtn_temp_list_valid,ignore_index=True)
    

    k=k+1

    #saves a temp version of the files
    #saving the file to an output
  
  
  #print("Of which",len(X_train),"were used for training and",len(X_test),"for validation")
  print("Saving temp files")
  df_metrics.to_csv("/content/drive/My Drive/RTM_Inversion/Pure/Run_02/Tables/Optim_Temp_KFold_TrainingError_csv2_Corrected_500epoch_1750.csv",sep=";",decimal=",")
  df_metrics_valid.to_csv("/content/drive/My Drive/RTM_Inversion/Pure/Run_02/Tables/Optim_Temp_KFold_OutOfBag_csv2_Corrected_500epoch_1750.csv",sep=";",decimal=",")

  df_metrics.to_csv("/content/drive/My Drive/RTM_Inversion/Pure/Run_02/Tables/Optim_Temp_KFold_TrainingError_csv1_Corrected_500epoch_1750.csv",sep=",",decimal=".")
  df_metrics_valid.to_csv("/content/drive/My Drive/RTM_Inversion/Pure/Run_02/Tables/Optim_Temp_KFold_OutOfBag_csv1_Corrected_500epoch_1750.csv",sep=",",decimal=".")


#saving the file to an output
df_metrics.to_csv("/content/drive/My Drive/RTM_Inversion/Pure/Run_02/Tables/Optim_KFold_TrainingError_csv2_Corrected_500epoch_1750.csv",sep=";",decimal=",")
df_metrics_valid.to_csv("/content/drive/My Drive/RTM_Inversion/Pure/Run_02/Tables/Optim_KFold_OutOfBag_csv2_Corrected_500epoch_1750.csv",sep=";",decimal=",")

df_metrics.to_csv("/content/drive/My Drive/RTM_Inversion/Pure/Run_02/Tables/Optim_KFold_TrainingError_csv1_Corrected_500epoch_1750.csv",sep=",",decimal=".")
df_metrics_valid.to_csv("/content/drive/My Drive/RTM_Inversion/Pure/Run_02/Tables/Optim_KFold_OutOfBag_csv1_Corrected_500epoch_1750.csv",sep=",",decimal=".")

Loop for the following set of LHS samples: [1750, 2000, 2500, 3000, 3500, 4000]
Generating trait table for 1750 samples
Of which, 10% were taken as out-of-bag: 175
processing fold nr: 1
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
processing fold nr: 2
processing fold nr: 3
processing fold nr: 4
processing fold nr: 5
Saving temp files
Generating trait table for 2000 samples
Of which, 10% were taken as out-of-bag: 200
processing fold nr: 1
processing fold nr: 2
processing fold nr: 3
processing fold nr: 4
processing fold nr: 5
Saving temp files
Generating trait table for 2500 samples
Of which, 10% were taken as out-of-bag: 250
processing fold nr: 1
processing fold nr: 2
processing fold nr: 3
processing fold nr: 4
processing fold nr: 5
Saving temp files
Generating trait table for 3000 samples
Of which, 10% were taken as out-of-bag: 300
processing fold nr: 1
processing fold nr: 2
processing fold nr: 3
processing fold nr: 4
processing fold nr: 5
Saving te